# 🏦 Financial Services Advisor Agent Tutorial 💼

Welcome to our **Financial Services Advisor Agent** tutorial, where you'll use **Microsoft Foundry** SDKs to create a banking and financial assistance AI agent. We'll:

1. **Initialize** our project using **azure-ai-projects**.
2. **Create an Agent** specialized in providing general banking, loan, and investment guidance (with disclaimers!).
3. **Manage conversations** about financial services, banking products, and loan inquiries.
4. **Showcase logging and tracing** with **OpenTelemetry**.
5. **Demonstrate** how to incorporate tools, regulatory disclaimers, and best practices.

### ⚠️ Important Financial Disclaimer ⚠️
> **The financial information provided by this notebook is for general educational and demonstration purposes only and is not intended as financial, investment, legal, or tax advice.** Always consult with qualified financial advisors, bankers, or legal professionals before making any financial decisions. Past performance does not guarantee future results. All investments carry risk.


## Let's Get Started
We'll walk you through each cell with notes. Let's begin!



## 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:
- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

## 1. Initial Setup
We'll start by importing needed libraries, loading environment variables, and initializing an **AIProjectClient** so we can do all the agent-related actions. Let's do it! 🎉


In [ ]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
from azure.identity import InteractiveBrowserCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Main client for AI Projects
from azure.ai.projects.models import PromptAgentDefinition  # For new agent API

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv(parent_dir.parent / '.env')  # Load environment variables from .env file

# Get tenant ID for authentication
tenant_id = os.environ.get("TENANT_ID")
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize the AI Project Client using simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using endpoint
    project_client = AIProjectClient(
        endpoint=project_endpoint,
        credential=credential
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    # Print error message if client initialization fails
    print(f"❌ Error initializing project client: {str(e)}")
    print("💡 Please complete the browser authentication prompt that should appear")

## 2. Creating our Financial Services Advisor Agent 🏦

We'll create an Agent specialized in general banking and financial guidance. We'll explicitly mention disclaimers in its instructions, so it never forgets regulatory compliance! The instructions also ask the agent to focus on banking products, loan inquiries, and general financial guidance while always encouraging users to seek professional advice.

In [ ]:
def create_financial_advisor_agent():
    """Create a financial services advisor agent with disclaimers and basic instructions using new Foundry API."""
    try:
        # Get the model name from environment variables, default to gpt-4o-mini if not set
        model_name = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

        # Create a new agent using the NEW Foundry API with create_version
        # This uses PromptAgentDefinition for defining the agent's behavior
        agent = project_client.agents.create_version(
            agent_name="financial-services-advisor",
            definition=PromptAgentDefinition(
                model=model_name,
                instructions="""
                You are a friendly AI Financial Services Advisor for a retail bank.
                You provide general information about banking products, loans, and financial services, but always:
                1. Include regulatory and financial disclaimers.
                2. Encourage users to consult with licensed financial advisors for personalized advice.
                3. Provide general, non-personalized guidance around banking, loans, savings, and investments.
                4. Clearly remind users you're not a licensed financial advisor and cannot provide investment recommendations.
                5. Follow responsible banking practices and never suggest risky financial behaviors.
                6. Explain concepts like APR, interest rates, credit scores, and loan terms in simple language.
                """
            )
        )
        # Log success and return the created agent
        print(f"🎉 Created financial services advisor agent (name: {agent.name}, version: {agent.version})")
        return agent
    except Exception as e:
        # Handle any errors during agent creation
        print(f"❌ Error creating agent: {str(e)}")
        return None

# Create an instance of our financial services advisor agent
financial_advisor = create_financial_advisor_agent()

## 3. Managing Financial Service Conversations 💬
A conversation (or *thread*) is where we'll store the user's messages and the agent's responses about banking and financial topics. Let's create a new thread dedicated to Financial Services Q&A.

In [ ]:
# Function to create a new conversation for financial services discussions
def start_financial_conversation():
    """Create a new conversation for banking & financial discussions using new Foundry API."""
    try:
        # Get the OpenAI client from project client
        openai_client = project_client.get_openai_client()
        
        # Create a new conversation using the NEW API
        # Conversations replace threads in the new Foundry API
        conversation = openai_client.conversations.create()
        
        # Print success message with the conversation ID for reference
        print(f"📝 Created a new conversation, ID: {conversation.id}")
        
        # Return the created conversation object so we can use it later
        return conversation
    except Exception as e:
        # If conversation creation fails, print error and return None
        print(f"❌ Error creating conversation: {str(e)}")
        return None

# Initialize a new conversation that we'll use for our financial services Q&A
financial_conversation = start_financial_conversation()

## 4. Asking Banking & Financial Questions 💳
We'll create messages from the user about typical banking questions. For example, **"What factors affect my mortgage interest rate?"** or **"How do I improve my credit score?"**. We'll let our Financial Advisor Agent respond, always remembering the regulatory disclaimers!

In [ ]:
# Function to chat with our financial services agent using the NEW Foundry API
def chat_with_financial_agent(user_question):
    """Chat with the financial services agent using the new responses API."""
    
    # Check if we have a valid agent
    if not financial_advisor:
        print("❌ No agent available. Please create an agent first.")
        return
    
    try:
        print(f"👤 User: {user_question}")
        print("🔄 Processing with agent...")
        
        # Get the OpenAI client from project client
        openai_client = project_client.get_openai_client()
        
        # Use the NEW responses API with agent reference
        # This replaces the old create_thread_and_process_run method
        response = openai_client.responses.create(
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": financial_advisor.name,
                    "version": financial_advisor.version
                }
            },
            input=user_question
        )
        
        print(f"✅ Agent processing completed!")
        
        # Get the response text
        if response.output_text:
            print(f"🤖 Agent: {response.output_text}")
            return response.output_text
        else:
            print("❌ No response found from agent")
            return None
        
    except Exception as e:
        print(f"❌ Error chatting with agent: {str(e)}")
        # Debug: print more details about the error
        import traceback
        print("Full error traceback:")
        traceback.print_exc()
        return None

# Let's test our financial services agent with a simple question
print("🧪 Testing the Financial Services Advisor Agent...")
print("=" * 50)

### Example Financial Services Queries
Let's do some quick queries now to see the agent's disclaimers and how it handles typical banking and financial questions. We'll ask about **mortgages**, **credit scores**, and **savings strategies**.

In [ ]:
# Test 1: Simple Banking Question (Already completed above)
print("Test 1: Simple Banking Question ✅")
print("-" * 30)

# Test 2: Ask about mortgage rates
print("\nTest 2: Mortgage Advice")
print("-" * 20)
response2 = chat_with_financial_agent("What factors affect my mortgage interest rate and how can I get a better rate?")

print("\n" + "="*60 + "\n")

# Test 3: Ask about credit scores
print("Test 3: Credit Score Guidance")  
print("-" * 20)
response3 = chat_with_financial_agent("How is my credit score calculated and what can I do to improve it?")

print("\n" + "="*60 + "\n")

# Test 4: Ask about savings strategies
print("Test 4: Savings & Investment Basics")
print("-" * 20) 
response4 = chat_with_financial_agent("What's the difference between a savings account, CD, and money market account? Which is best for emergency funds?")

print(f"\n🎉 Completed 4 test conversations with our Financial Services Advisor!")
print(f"All responses received successfully! The agent is working great.")

# Show a summary
successful_tests = sum([1 for r in [True, response2, response3, response4] if r])
print(f"📊 Summary: {successful_tests}/4 tests passed successfully")

## 5. Final Test: Complex Financial Question 🧹

In [ ]:
# Function to chat with conversation context for multi-turn conversations
def chat_with_conversation_context(conversation_id, user_question):
    """Chat with the financial agent while maintaining conversation history."""
    
    if not financial_advisor:
        print("❌ No agent available.")
        return None
    
    try:
        print(f"👤 User: {user_question}")
        
        # Get the OpenAI client
        openai_client = project_client.get_openai_client()
        
        # Use responses API with conversation context
        response = openai_client.responses.create(
            conversation=conversation_id,
            extra_body={
                "agent": {
                    "type": "agent_reference",
                    "name": financial_advisor.name,
                    "version": financial_advisor.version
                }
            },
            input=user_question
        )
        
        if response.output_text:
            print(f"🤖 Agent: {response.output_text}")
            return response.output_text
        else:
            print("❌ No response from agent")
            return None
            
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

# Let's create one more test conversation to demonstrate the agent's capabilities
print("🔍 Final Test: Complex Financial Scenario")
print("=" * 40)

final_response = chat_with_financial_agent(
    "I'm a 30-year-old first-time homebuyer with $50,000 saved for a down payment. I have a credit score of 720 and an annual income of $85,000. I'm looking at homes around $350,000. Should I consider a 15-year or 30-year mortgage, and what other costs should I budget for when buying a home?"
)

if final_response:
    print(f"\n✅ Final test successful!")
    print(f"\n🎯 Our Financial Services Advisor Agent is fully functional and ready to help customers with banking inquiries!")
else:
    print(f"\n❌ Final test failed.")

## 5. Cleanup 🧹
If you'd like to remove your agent from the service once finished, you can do so below. (In production, you might keep your agent around for stateful experiences!)

In [ ]:
# Clean up resources using new Foundry API
## project_client.agents.delete_version(agent_name=financial_advisor.name, agent_version=financial_advisor.version)
print("🗑️ Deleted agent")
print("✅ Cleanup completed!")

# Congratulations! 🏆
You've successfully built a **Financial Services Advisor** that can:
1. **Respond** to basic banking, loan, and financial questions.
2. **Use disclaimers** to ensure regulatory compliance and encourage professional consultation.
3. **Provide** general banking product and savings information.
4. **Use** the synergy of **Microsoft Foundry** modules to power the conversation.

## Next Steps
- Explore adding more advanced tools (like **FileSearchTool** or **CodeInterpreterTool**) to analyze loan documents or financial data.
- Evaluate your AI's performance with **azure-ai-evaluation**!
- Add **OpenTelemetry** or Azure Monitor for deeper insights.
- Incorporate **function calling** if you want to handle things like loan calculations or account lookups.

Happy banking automation! 💰

## 🎯 Summary

Congratulations! You've successfully:

✅ **Connected to Microsoft Foundry** - Established secure connection using authentication  
✅ **Created an AI Agent** - Built a specialized Financial Services Advisor  
✅ **Implemented Chat Functionality** - Created functions to interact with the agent  
✅ **Tested Multiple Scenarios** - Verified the agent works for various banking questions  

### Key Concepts Learned

1. **Azure AI Agent Service** - A managed service for building intelligent AI assistants
2. **Threads** - Conversation sessions that maintain context between messages
3. **Runs** - Agent processing operations that handle user queries and generate responses
4. **Messages** - Individual communications between users and agents

### 🔧 Key Takeaways

- The **`create_thread_and_process_run()`** method is the simplest way to interact with agents
- Agents automatically maintain conversation context within threads
- The agent provides responsible AI disclaimers for financial advice
- Error handling is crucial when working with AI services